<a href="https://colab.research.google.com/github/vhmartinsp/yolo_darknet_deteccao/blob/main/treinamento_rede_yolo_coco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
from collections import defaultdict
from pycocotools.coco import COCO
from google.colab import drive

# Montando o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Definição de diretórios
base_dir = '/content/drive/MyDrive/darknet'
os.makedirs(base_dir, exist_ok=True)
backup_dir = os.path.join(base_dir, 'backup')
os.makedirs(backup_dir, exist_ok=True)

cfg_file_path = "/content/yolov3.cfg"
coco_names_path = "/content/coco.names"
coco_data_path = "/content/coco.data"
train_txt_path = "/content/train.txt"
val_txt_path = "/content/val.txt"
test_txt_path = "/content/test.txt"

In [ ]:
# Verificando instalação do CUDA
def check_cuda():
    os.system("/usr/local/cuda/bin/nvcc --version")

check_cuda()

# # Instalando cuDNN
# os.system("tar -xzvf /content/drive/MyDrive/darknet/cuDNN/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/")
# os.system("chmod a+r /usr/local/cuda/include/cudnn.h")

# # Clonando e compilando Darknet
# if not os.path.exists('/content/darknet'):
#     os.system("git clone https://github.com/kriyeng/darknet /content/darknet")

# os.chdir('/content/darknet')
# os.system("make")

# # Copiando Darknet compilado para o Google Drive
# os.system(f"cp ./darknet {base_dir}/bin/darknet")
# os.system("chmod +x ./darknet")


0

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb
!dpkg -i cuda-keyring_1.1-1_all.deb
!apt-get update
!apt-get install -y libcudnn8 libcudnn8-dev


--2025-02-09 02:28:28--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 2.16.6.17, 2.16.6.12
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|2.16.6.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4332 (4.2K) [application/x-deb]
Saving to: ‘cuda-keyring_1.1-1_all.deb.4’

cuda-keyring_1.1-1_ 100%[===================>]   4.23K  --.-KB/s    in 0s      

2025-02-09 02:28:28 (1.41 GB/s) - ‘cuda-keyring_1.1-1_all.deb.4’ saved [4332/4332]

(Reading database ... 124944 files and directories currently installed.)
Preparing to unpack cuda-keyring_1.1-1_all.deb ...
Unpacking cuda-keyring (1.1-1) over (1.1-1) ...
Setting up cuda-keyring (1.1-1) ...
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backpo

In [ ]:
!mkdir -p /content/drive/MyDrive/darknet


In [ ]:
!ls -lh /content/drive/MyDrive


total 85K
drwx------ 2 root root 4.0K Dec 21 14:29  Backup
drwx------ 2 root root 4.0K Apr 11  2024  Backups
drwx------ 2 root root 4.0K Apr 22  2024  Captures
drwx------ 2 root root 4.0K Jan 19 21:41 'Cerfiicados em dados'
drwx------ 2 root root 4.0K Apr 11  2024  Certificados
drwx------ 7 root root 4.0K Feb  8 23:53  coco2017
drwx------ 2 root root 4.0K Apr 11  2024 'Colab Notebooks'
drwx------ 2 root root 4.0K Apr 21  2023  Contratos
drwx------ 2 root root 4.0K Apr 11  2024 'Criação de Dashboards com Looker Studio (2023)'
drwx------ 2 root root 4.0K Apr 11  2024  Currículo
drwx------ 2 root root 4.0K Feb  9 02:28  darknet
drwx------ 2 root root 4.0K Feb  8 20:26  darknet_data
drwx------ 2 root root 4.0K Feb  8 20:26  darknet_weights
drwx------ 2 root root 4.0K Apr 22  2024  Data
drwx------ 2 root root 4.0K Apr 11  2024  Estatística+para+Ciência+de+Dados+e+Machine+Learning
drwx------ 2 root root 4.0K Apr 11  2024 'ETL Python'
drwx------ 2 root root 4.0K Jul  7  2024  Formacao.An

In [ ]:
!rm -rf /content/drive/MyDrive/darknet
!git clone https://github.com/AlexeyAB/darknet.git /content/drive/MyDrive/darknet


In [35]:
%cd /content/drive/MyDrive/darknet


/content/drive/MyDrive/darknet


Como darknet sem extensão nao apareceu, irei compilá-lo

In [37]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile


In [38]:
!make -j$(nproc)


mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: In member function ‘bool JSON_sender::write(const char*)’:
./src/http_stream.cpp:253:21: warning: unused variable ‘n’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-

Verificando se a compilação foi bem-sucedida

In [39]:
!ls -lh


total 5.0M
drwx------ 5 root root 4.0K Feb  9 02:29 3rdparty
drwx------ 2 root root 4.0K Feb  9 02:34 backup
drwx------ 3 root root 4.0K Feb  9 02:29 build
-rw------- 1 root root  44K Feb  9 02:29 build.ps1
drwx------ 3 root root 4.0K Feb  9 02:29 cfg
drwx------ 3 root root 4.0K Feb  9 02:29 cmake
-rw------- 1 root root  30K Feb  9 02:29 CMakeLists.txt
-rwx------ 1 root root 4.8M Feb  9 02:36 darknet
-rw------- 1 root root 1.5K Feb  9 02:29 DarknetConfig.cmake.in
-rw------- 1 root root 9.4K Feb  9 02:29 darknet_images.py
-rw------- 1 root root  11K Feb  9 02:29 darknet.py
-rw------- 1 root root 7.9K Feb  9 02:29 darknet_video.py
drwx------ 3 root root 4.0K Feb  9 02:29 data
-rw------- 1 root root  366 Feb  9 02:29 docker-compose.yml
-rw------- 1 root root  774 Feb  9 02:29 Dockerfile.cpu
-rw------- 1 root root  834 Feb  9 02:29 Dockerfile.gpu
-rwx------ 1 root root  110 Feb  9 02:29 image_yolov3.sh
-rwx------ 1 root root  110 Feb  9 02:29 image_yolov4.sh
drwx------ 2 root root 4.0K Feb

In [40]:
!chmod +x darknet


In [42]:
!ls -lh /usr/local/cuda/lib64/


total 6.7G
drwxr-xr-x 6 root root 4.0K Jul 10  2024 cmake
lrwxrwxrwx 1 root root   19 Jun  6  2024 libaccinj64.so -> libaccinj64.so.12.5
lrwxrwxrwx 1 root root   22 Jun  6  2024 libaccinj64.so.12.5 -> libaccinj64.so.12.5.82
-rw-r--r-- 1 root root 2.4M Jun  6  2024 libaccinj64.so.12.5.82
-rw-r--r-- 1 root root 1.5M Jun  6  2024 libcheckpoint.so
lrwxrwxrwx 1 root root   17 Jun  6  2024 libcublasLt.so -> libcublasLt.so.12
lrwxrwxrwx 1 root root   23 Jun  6  2024 libcublasLt.so.12 -> libcublasLt.so.12.5.3.2
-rw-r--r-- 1 root root 427M Jun  6  2024 libcublasLt.so.12.5.3.2
-rw-r--r-- 1 root root 685M Jun  6  2024 libcublasLt_static.a
lrwxrwxrwx 1 root root   15 Jun  6  2024 libcublas.so -> libcublas.so.12
lrwxrwxrwx 1 root root   21 Jun  6  2024 libcublas.so.12 -> libcublas.so.12.5.3.2
-rw-r--r-- 1 root root 100M Jun  6  2024 libcublas.so.12.5.3.2
-rw-r--r-- 1 root root 167M Jun  6  2024 libcublas_static.a
-rw-r--r-- 1 root root 1.6M Jun  6  2024 libcudadevrt.a
lrwxrwxrwx 1 root root   15 Ju

In [47]:
!apt-get install cuda-drivers


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  cpp-12 cuda-drivers-570 dctrl-tools dkms fakeroot gcc-12 keyboard-configuration libasan8
  libfakeroot libfontenc1 libgail-common libgail18 libgcc-12-dev libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libgudev-1.0-0 libjansson4 liblocale-gettext-perl libnvidia-cfg1-570
  libnvidia-common-570 libnvidia-compute-570 libnvidia-decode-570 libnvidia-encode-570
  libnvidia-extra-570 libnvidia-fbc1-570 libnvidia-gl-570 librsvg2-common libtsan2 libxcvt0
  libxfont2 libxkbfile1 nvidia-compute-utils-570 nvidia-dkms-570 nvidia-driver-570
  nvidia-firmware-570-570.86.15 nvidia-kernel-common-570 nvidia-kernel-source-570 nvidia-modprobe
  nvidia-settings nvidia-utils-570 python3-xkit screen-resolution-extra switcheroo-control
  systemd-hwe-hwdb udev x11-xkb-utils xcvt xfonts-base xfonts-encodings xfonts-utils xserver-common
  xserver-xorg-core xserver-x

In [48]:
!./darknet


usage: ./darknet <function>


In [49]:
# Caminhos do dataset COCO
base_dir = "/content/drive/MyDrive/coco2017"
train_dir = os.path.join(base_dir, "train2017")
val_dir = os.path.join(base_dir, "val2017")
test_dir = os.path.join(base_dir, "test2017")  # Teste não tem anotações
annotation_dir = os.path.join(base_dir, "annotations")

train_annotation_file = os.path.join(annotation_dir, "instances_train2017.json")
val_annotation_file = os.path.join(annotation_dir, "instances_val2017.json")

# Carregando anotações do COCO
coco_train = COCO(train_annotation_file)

loading annotations into memory...
Done (t=27.78s)
creating index...
index created!


In [50]:

# Analisando classes do COCO
category_count = defaultdict(int)
category_mapping = {cat["id"]: cat["name"] for cat in coco_train.dataset["categories"]}


for annotation in coco_train.dataset["annotations"]:
    category_count[annotation["category_id"]] += 1

print(f"\nNúmero total de classes: {len(category_mapping)}")
for category_id, category_name in category_mapping.items():
    count = category_count[category_id]
    print(f"ID: {category_id} - Nome: {category_name} - Instâncias: {count}")




Número total de classes: 80
ID: 1 - Nome: person - Instâncias: 262465
ID: 2 - Nome: bicycle - Instâncias: 7113
ID: 3 - Nome: car - Instâncias: 43867
ID: 4 - Nome: motorcycle - Instâncias: 8725
ID: 5 - Nome: airplane - Instâncias: 5135
ID: 6 - Nome: bus - Instâncias: 6069
ID: 7 - Nome: train - Instâncias: 4571
ID: 8 - Nome: truck - Instâncias: 9973
ID: 9 - Nome: boat - Instâncias: 10759
ID: 10 - Nome: traffic light - Instâncias: 12884
ID: 11 - Nome: fire hydrant - Instâncias: 1865
ID: 13 - Nome: stop sign - Instâncias: 1983
ID: 14 - Nome: parking meter - Instâncias: 1285
ID: 15 - Nome: bench - Instâncias: 9838
ID: 16 - Nome: bird - Instâncias: 10806
ID: 17 - Nome: cat - Instâncias: 4768
ID: 18 - Nome: dog - Instâncias: 5508
ID: 19 - Nome: horse - Instâncias: 6587
ID: 20 - Nome: sheep - Instâncias: 9509
ID: 21 - Nome: cow - Instâncias: 8147
ID: 22 - Nome: elephant - Instâncias: 5513
ID: 23 - Nome: bear - Instâncias: 1294
ID: 24 - Nome: zebra - Instâncias: 5303
ID: 25 - Nome: giraffe - I

In [51]:
# Criando coco.names
with open(coco_names_path, "w") as f:
    for category_name in category_mapping.values():
        f.write(f"{category_name}\n")

In [52]:


# Ajustando yolov3.cfg
num_classes = len(category_mapping)
filters_value = (num_classes + 5) * 3  # Fórmula correta para filters

with open(cfg_file_path, "r") as file:
    cfg_data = file.readlines()

for i in range(len(cfg_data)):
    if "classes=" in cfg_data[i]:
        cfg_data[i] = f"classes={num_classes}\n"
    if "filters=" in cfg_data[i - 1]:  # Linha antes de cada 'yolo'
        cfg_data[i - 1] = f"filters={filters_value}\n"

with open(cfg_file_path, "w") as file:
    file.writelines(cfg_data)

In [53]:
# Função para filtrar imagens existentes
def filter_coco_annotations(coco, image_dir):
    available_images = set(os.listdir(image_dir))
    valid_images = set()

    for img_info in coco.dataset["images"]:
        img_filename = img_info["file_name"]
        if img_filename in available_images:
            valid_images.add(img_filename)

    return valid_images

# Filtrando imagens existentes no conjunto de treino
train_valid_images = filter_coco_annotations(coco_train, train_dir)

# Criando train.txt
def generate_train_list(image_dir, valid_images, output_file):
    with open(output_file, "w") as f:
        for img_name in sorted(valid_images):
            f.write(f"{os.path.join(image_dir, img_name)}\n")

generate_train_list(train_dir, train_valid_images, train_txt_path)

In [54]:
# Criando coco.data
coco_data_content = f"""
classes={num_classes}
train={train_txt_path}
valid={val_txt_path}
names={coco_names_path}
backup={backup_dir}/
"""

with open(coco_data_path, "w") as f:
    f.write(coco_data_content)

# Baixando pesos do YOLO
os.system(f"wget https://pjreddie.com/media/files/yolov3.weights -O {backup_dir}/yolov3_final.weights")

# Iniciando treinamento do YOLO
os.system(f"./darknet detector train {coco_data_path} {cfg_file_path} {backup_dir}/yolov3_final.weights")

# Função para gerar test.txt corretamente
def filter_test_images(image_dir):
    available_images = set(os.listdir(image_dir))
    valid_images = sorted(available_images)
    return valid_images

test_valid_images = filter_test_images(test_dir)

def generate_test_list(image_dir, valid_images, output_file):
    with open(output_file, "w") as f:
        for img_name in valid_images:
            f.write(f"{os.path.join(image_dir, img_name)}\n")

generate_test_list(test_dir, test_valid_images, test_txt_path)

# Executando detecção no conjunto de teste
os.system(f"./darknet detector test {coco_data_path} {cfg_file_path} {backup_dir}/yolov3_final.weights -dont_show < {test_txt_path}")

256

In [56]:
os.system(f"./darknet detector test {coco_data_path} {cfg_file_path} {backup_dir}/yolov3_final.weights -dont_show -ext_output < {test_txt_path} > results.txt")


256

In [57]:
with open(test_txt_path, "r") as f:
    lines = f.readlines()
    print("🔍 Primeiras 5 linhas do test.txt:")
    print(lines[:5])


🔍 Primeiras 5 linhas do test.txt:
['/content/drive/MyDrive/coco2017/test2017/000000000001.jpg\n', '/content/drive/MyDrive/coco2017/test2017/000000000016.jpg\n', '/content/drive/MyDrive/coco2017/test2017/000000000019.jpg\n', '/content/drive/MyDrive/coco2017/test2017/000000000057.jpg\n', '/content/drive/MyDrive/coco2017/test2017/000000000063.jpg\n']


In [59]:
!./darknet detector test {coco_data_path} {cfg_file_path} {backup_dir}/yolov3_final.weights -dont_show -ext_output -i 0 -thresh 0.25 < {test_txt_path}


CUDA status Error: file: ./src/dark_cuda.c: func: cuda_set_device() line: 38

 CUDA Error: no CUDA-capable device is detected
Darknet error location: ./src/dark_cuda.c, cuda_set_device(), line #38
CUDA Error: no CUDA-capable device is detected: Bad file descriptor
backtrace (8 entries)
1/8: ./darknet(log_backtrace+0x38) [0x58b1366aa1f8]
2/8: ./darknet(error+0x3d) [0x58b1366aa2dd]
3/8: ./darknet(check_error+0xd0) [0x58b1366acbd0]
4/8: ./darknet(check_error_extended+0x7c) [0x58b1366accbc]
5/8: ./darknet(main+0x281) [0x58b1366696c1]
6/8: /lib/x86_64-linux-gnu/libc.so.6(+0x29d90) [0x7cfa56bd4d90]
7/8: /lib/x86_64-linux-gnu/libc.so.6(__libc_start_main+0x80) [0x7cfa56bd4e40]
8/8: ./darknet(_start+0x25) [0x58b13666b9e5]


In [55]:
!./darknet detector train cfg/obj.data cfg/yolov3_custom.cfg yolov3_custom_last.weights -dont_show


CUDA status Error: file: ./src/dark_cuda.c: func: cuda_set_device() line: 38

 CUDA Error: no CUDA-capable device is detected
Darknet error location: ./src/dark_cuda.c, cuda_set_device(), line #38
CUDA Error: no CUDA-capable device is detected: Bad file descriptor
backtrace (8 entries)
1/8: ./darknet(log_backtrace+0x38) [0x5bfb84e171f8]
2/8: ./darknet(error+0x3d) [0x5bfb84e172dd]
3/8: ./darknet(check_error+0xd0) [0x5bfb84e19bd0]
4/8: ./darknet(check_error_extended+0x7c) [0x5bfb84e19cbc]
5/8: ./darknet(main+0x281) [0x5bfb84dd66c1]
6/8: /lib/x86_64-linux-gnu/libc.so.6(+0x29d90) [0x7df8105d4d90]
7/8: /lib/x86_64-linux-gnu/libc.so.6(__libc_start_main+0x80) [0x7df8105d4e40]
8/8: ./darknet(_start+0x25) [0x5bfb84dd89e5]
